# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/will/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/will/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM disaster_response",engine) 
X = df.filter(items=['id', 'message', 'original', 'genre'])
# here we drop child allone because we dont have a single message that is classified as such
y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'],  axis=1).astype(float)
# transform all it on binary classification
y['related']=y['related'].map(lambda x: 1 if x == 2 else x)



df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(x).lower().strip() for x in tokens]
   

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:


pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier()) 
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train['message'],y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the accuracy, precision and recall on both the training set and the test set. You can use sklearn's `classification_report` function here. 

In [6]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])

In [7]:
from sklearn.metrics import classification_report
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print("********************************************************************************")
print(classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.90      0.86      4962
               request       0.83      0.35      0.49      1097
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.40      0.52      2652
          medical_help       0.44      0.01      0.02       508
      medical_products       0.88      0.05      0.09       307
     search_and_rescue       0.50      0.02      0.03       182
              security       0.00      0.00      0.00       119
              military       0.67      0.02      0.03       237
                 water       0.84      0.12      0.21       398
                  food       0.85      0.20      0.33       699
               shelter       0.93      0.06      0.12       572
              clothing       0.80      0.04      0.08        99
                 money       0.80      0.03      0.05       147
        missing_people       0.00      

/home/will/.local/share/virtualenvs/scene-variations-ZwQ6vwJV/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__max_depth': [10, 20, 30],
    'clf__max_features': ['auto', 'sqrt'],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__min_samples_split': [2, 5, 10],
    'clf__n_estimators': [10, 20, 40]}



cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro',verbose= 1,n_jobs =-1)
cv.fit(X_train['message'], y_train)

Fitting 3 folds for each of 5832 candidates, totalling 17496 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 30.4min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 48.5min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 70.9min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 96.4min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 126.1min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 159.5min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 196.7min


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

In [ ]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print("********************************************************************************")
print(classification_report(y_train.values, y_pred_train, target_names=y.columns.values))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.